In [ ]:
# Conv and Vit models for image classification

import pandas as pd
import numpy as np
import os
import torchvision.datasets as datasets
import torch
from torchvision.transforms import v2
import torchvision.transforms.functional as TF

# Transform 224 224
train_transform_224 = v2.Compose([
    v2.RandomResizedCrop(224),
    v2.RandomHorizontalFlip(),
    v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
])

test_transform_224 = v2.Compose([
    v2.Resize(256),
    v2.CenterCrop(224),
    v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Transform inceptionv3 299 299
train_transform_299 = v2.Compose([
    v2.RandomResizedCrop(299),
    v2.RandomHorizontalFlip(),
    v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
])

test_transform_299 = v2.Compose([
    v2.Resize(299),
    v2.CenterCrop(299),
    v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Transform 256 256
train_transform_256 = v2.Compose([
    v2.RandomResizedCrop(256),
    v2.RandomHorizontalFlip(),
    v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
])

test_transform_256 = v2.Compose([
    v2.Resize(256),
    v2.CenterCrop(256),
    v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Data 

train_dataset_224 = datasets.ImageFolder('../Datasets/calc/train', transform=train_transform_224)
test_dataset_224 = datasets.ImageFolder('../Datasets/calc/test', transform=test_transform_224)

train_dataset_299 = datasets.ImageFolder("../Datasets/calc/train", transform=train_transform_299)
test_dataset_299 = datasets.ImageFolder('../Datasets/calc/test', transform=test_transform_299)

train_dataset_256 = datasets.ImageFolder('../Datasets/calc/train', transform=train_transform_256)
test_dataset_256 = datasets.ImageFolder('../Datasets/calc/test', transform=test_transform_256)


# Models

import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

## ResNet-50
resnet_model = resnet50(weights='IMAGENET1K_V2')
### Binary Classification
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2)
resnet_model = resnet_model.to('cuda')

## InceptionV3
from torchvision.models import inception_v3

inception_model = inception_v3(weights='IMAGENET1K_V1')
### Binary Classification
inception_model.fc = nn.Linear(inception_model.fc.in_features, 2)
inception_model = inception_model.to('cuda')

## Swin Transformer
from torchvision.models import swin_b

swin_model = swin_b(weights='IMAGENET1K_V1')
### Binary Classification
swin_model.head = nn.Linear(swin_model.head.in_features, 2)
swin_model = swin_model.to('cuda')

## ViT

from torchvision.models import vit_b_16

vit_model = vit_b_16(weights='IMAGENET1K_V1')
### Binary Classification
vit_model.heads.head = nn.Linear(vit_model.heads.head.in_features, 2)
vit_model = vit_model.to('cuda')
# Training

from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import LinearWarmupScheduler 
from torch.optim.lr_scheduler import *
# Hyperparameter Tuning
### Learning Rate
lr = 0.001
### Batch Size
batch_size = 32
### Number of Epochs
num_epochs = 30

# Loss Function and Optimizer
### AdamW
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(resnet_model.parameters(), lr=lr)
# Linear Warmup scheduler
scheduler = LinearWarmupScheduler(optimizer, warmup_period=10, warmup_start_lr=0, num_training_steps=num_epochs)

# Cosine Annealing LR
#scheduler = CosineAnnealingLR(optimizer, num_epochs)

# Data Loader

train_loader_224 = DataLoader(train_dataset_224, batch_size=batch_size, shuffle=True)
test_loader_224 = DataLoader(test_dataset_224, batch_size=batch_size, shuffle=False)

train_loader_299 = DataLoader(train_dataset_299, batch_size=batch_size, shuffle=True)
test_loader_299 = DataLoader(test_dataset_299, batch_size=batch_size, shuffle=False)

train_loader_256 = DataLoader(train_dataset_256, batch_size=batch_size, shuffle=True)
test_loader_256 = DataLoader(test_dataset_256, batch_size=batch_size, shuffle=False)

# Training Loop for ResNet-50
for epoch in range(num_epochs):
    resnet_model.train()
    for x, y in train_loader_224:
        x, y = x.to('cuda'), y.to('cuda')
        optimizer.zero_grad()
        y_pred = resnet_model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
    scheduler.step()
    resnet_model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in test_loader_224:
            x, y = x.to('cuda'), y.to('cuda')
            y_pred = resnet_model(x)
            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        accuracy = correct / total
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

# Training Loop for InceptionV3
for epoch in range(num_epochs):
    inception_model.train()
    for x, y in train_loader_299:
        x, y = x.to('cuda'), y.to('cuda')
        optimizer.zero_grad()
        y_pred = inception_model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
    scheduler.step()
    inception_model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in test_loader_299:
            x, y = x.to('cuda'), y.to('cuda')
            y_pred = inception_model(x)
            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        accuracy = correct / total
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

# Training Loop for Swin Transformer
for epoch in range(num_epochs):
    swin_model.train()
    for x, y in train_loader_256:
        x, y = x.to('cuda'), y.to('cuda')
        optimizer.zero_grad()
        y_pred = swin_model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
    scheduler.step()
    swin_model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in test_loader_256:
            x, y = x.to('cuda'), y.to('cuda')
            y_pred = swin_model(x)
            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        accuracy = correct / total
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

# Training Loop for ViT
for epoch in range(num_epochs):
    vit_model.train()
    for x, y in train_loader_256:
        x, y = x.to('cuda'), y.to('cuda')
        optimizer.zero_grad()
        y_pred = vit_model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
    scheduler.step()
    vit_model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in test_loader_256:
            x, y = x.to('cuda'), y.to('cuda')
            y_pred = vit_model(x)
            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        accuracy = correct / total
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

# Save Models
torch.save(resnet_model.state_dict(), 'resnet_model.pth')
torch.save(inception_model.state_dict(), 'inception_model.pth')
torch.save(swin_model.state_dict(), 'swin_model.pth')
torch.save(vit_model.state_dict(), 'vit_model.pth')

